In [65]:
using JuMP, Cbc, Gurobi, NamedArrays, LinearAlgebra

# Constants
num_weeks = 21
consecutive_weight = 2

# Sets
P = [:NM, :MP, :SA, :LKC, :DG] # Pathologists
R = [:MC, :AK, :CC, :KB]
A = union(P,R)
W = 1:num_weeks

# Availability
availability = [
    0 0 0 1 0 0 1 0 0;
    0 1 1 1 0 0 0 0 1;
    0 0 1 1 0 0 0 0 1;
    0 1 1 1 0 0 0 0 0;
    0 0 1 1 0 0 0 1 0;
    0 0 0 1 0 0 1 1 0;
    0 1 0 1 1 0 0 0 0;
    0 1 0 1 1 0 0 0 1;
    0 0 1 0 1 0 0 0 1;
    0 0 1 1 0 0 1 0 0;
    0 0 1 1 0 0 1 0 0;
    0 0 1 1 0 0 1 0 0;
    0 0 0 1 0 1 1 0 0;
    0 0 0 1 0 1 0 0 0;
    0 0 0 0 1 0 0 0 0;
    1 1 0 0 1 0 0 0 0;
    0 1 0 0 0 0 0 0 1;
    0 1 0 1 0 0 0 0 1;
    0 1 0 1 0 0 0 0 0;
    0 0 0 0 1 0 1 0 0;
    0 0 0 1 0 1 1 0 0
]
t = NamedArray(availability, (W,A), ("Week", "Employee"))


# Parameters
p_max = round(0.46 * num_weeks) # 24/52 = 0.46
r_min = round(0.42 * num_weeks) # 22/52 = 0.42
r_max = round(0.48 * num_weeks) # 25/52 = 0.48

# Set constant to compute penalties and incentivize N/B balance
p_Nx_ideal = p_max / 2
p_Bx_ideal = p_max / 2

r_Nx_ideal = round(0.23 * num_weeks) # 12/52
r_Bx_ideal = round(0.23 * num_weeks) # 12/52

# Modeling
m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "TimeLimit", 20)
set_optimizer_attribute(m, "Presolve", 0)

# m = Model(NLopt.Optimizer)
# set_optimizer_attribute(m, "algorithm", :LD_MMA)

# Necropsies
@variable(m, n[W,A], Bin)

# Biopsies
@variable(m, b[W,A], Bin)

# Pathologist/resident target penalties for Nx & Bx
p_bal_penalty = sum((sum(n[w,p] for w in W) - p_Nx_ideal)^2 + (sum(b[w,p] for w in W) - p_Bx_ideal)^2 for p in P)
r_bal_penalty = sum((sum(n[w,r] for w in W) - r_Nx_ideal)^2 + (sum(b[w,r] for w in W) - r_Bx_ideal)^2 for r in R)

# Penalty for consecutive weeks
bb_penalty = sum(sum([b[w,a] for w in W[1:end-1]] .* [b[w,a] for w in W[2:end]]) for a in A)
bn_penalty = sum(sum([b[w,a] for w in W[1:end-1]] .* [n[w,a] for w in W[2:end]]) for a in A)
nn_penalty = sum(sum([n[w,a] for w in W[1:end-1]] .* [n[w,a] for w in W[2:end]]) for a in A)

consecutive_penalties = bb_penalty + bn_penalty + nn_penalty

#@expression(m, triplePenalty, sum(sum((b[w,a] + n[w,a]) * (b[w+1,a] + n[w+1,a]) * (b[w+2,a] + n[w+2,a]) for w in W[1:end-2]) for a in A))
@objective(m, Min, p_bal_penalty + r_bal_penalty + consecutive_weight + 3 * nn_penalty + 2 * bn_penalty + bb_penalty)

# Pathology limits
# @constraint(m, pLimit[p in P], sum(n[w,p] + b[w,p] for w in W) <= p_max)

# Resident limits
#@constraint(m, rLimit[r in R], r_min <= sum(n[w,r] + b[w,r] for w in W) <= r_max)

# Minimum staff for each duty
@constraint(m, minNx[w in W], sum(n[w,a] for a in A) >= 1)
@constraint(m, minBx[w in W], sum(b[w,a] for a in A) >= 1)

# Availability
@constraint(m, oneDuty[w in W, a in A], n[w,a] + b[w,a] <= -t[w,a] + 1)

# Resident must be paired with pathologist per duty
@constraint(m, rNxPair[w in W], sum(n[w,r] for r in R) <= sum(n[w,p] for p in P))
@constraint(m, rBxPair[w in W], sum(b[w,r] for r in R)  <= sum(b[w,p] for p in P))

# Maximum 1 resident per duty
@constraint(m, rNxMax[w in W], sum(n[w,r] for r in R) <= 1)
@constraint(m, rBxMax[w in W], sum(b[w,r] for r in R) <= 1)

cons = zeros(num_weeks, num_weeks)
for i in axes(cons[1:end-2,:],1)
    cons[i,i:i+2] .= 1
end

@constraint(m, noTriple[a in A, w in W], dot(n[:,a] .+ b[:,a], cons[w,:]) <= 2)
#@constraint(m, noTriple2[a in A, w in W], (n[:,a] .+ b[:,a]) .* cons[w,:] <= 3)

JuMP.optimize!(m)
solution_summary(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-18
Set parameter TimeLimit to value 20
Set parameter Presolve to value 0
Set parameter TimeLimit to value 20
Set parameter Presolve to value 0
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 504 rows, 378 columns and 2328 nonzeros
Model fingerprint: 0xe9ccf252
Model has 4338 quadratic objective terms
Variable types: 0 continuous, 378 integer (378 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+01]
  QObjective range [2e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+00]
Variable types: 0 continuous, 4338 integer (4338 binary)

Root relaxation: objective -3.580000e+02, 820 iterations, 0.03 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

LoadError: MethodError: [0mCannot `convert` an object of type [92mMissing[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/twiceprecision.jl:262
[0m  convert(::Type{T}, [91m::AbstractChar[39m) where T<:Number at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/char.jl:185
[0m  convert(::Type{T}, [91m::CartesianIndex{1}[39m) where T<:Number at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/multidimensional.jl:136
[0m  ...

In [62]:
function print_balance(b, n, W, staff)
    for a in staff
        bsum = sum(b[w,a] for w in W)
        nsum = sum(n[w,a] for w in W)
        println(a," ",value(nsum)," ",value(bsum))
    end
end

print_balance(b, n, W, A)

NM 5.0 5.0
MP 5.0 5.0
SA 5.0 4.0
LKC 2.0 2.0
DG 5.0 5.0
MC 5.0 4.0
AK 4.0 3.0
CC 4.0 4.0
KB 5.0 4.0


In [63]:
# CONSTRAINT TASKS

# [TO-DO] Maximize N/B/OFF cycles
# [TO-DO] Incorporate running balance from previous period (This will be important for recomputing schedule)
# [TO-DO] Insert RARC in 2 week chunks for residents

# [DONE] Time-off 
# [DONE] Limit on number of N/B resident 
# [DONE] Limit on number of N/B pathologist
# [DONE] Balance # N/B rotations

# Necropsy printed as -1 if scheduled, Biopsy printed +1
value.(-n .+ b), value(consecutive_penalties)

(2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:21
    Dimension 2, [:NM, :MP, :SA, :LKC, :DG, :MC, :AK, :CC, :KB]
And data, a 21×9 Matrix{Float64}:
  0.0   1.0  -1.0   0.0   0.0  -1.0   0.0   0.0   1.0
 -1.0   0.0   0.0   0.0  -1.0   0.0  -1.0   0.0   0.0
  0.0  -1.0   0.0   0.0   1.0   1.0   0.0  -1.0   0.0
 -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0
  1.0   0.0   0.0   0.0  -1.0  -1.0   1.0   0.0   0.0
  0.0  -1.0   1.0   0.0   1.0   1.0   0.0   0.0  -1.0
 -1.0   0.0   0.0   0.0   0.0   0.0  -1.0   0.0   0.0
  1.0   0.0  -1.0   0.0   0.0  -1.0   0.0   0.0   0.0
  0.0   1.0   0.0  -1.0   0.0   0.0  -1.0   1.0   0.0
  0.0   1.0   0.0   0.0  -1.0  -1.0   0.0   0.0   1.0
  1.0   0.0   0.0   0.0  -1.0   0.0   0.0   0.0   1.0
  0.0  -1.0   0.0   0.0   0.0   0.0   0.0  -1.0   0.0
  1.0  -1.0   1.0   0.0   0.0   0.0   0.0   1.0  -1.0
  0.0   0.0  -1.0   0.0   1.0   0.0  -1.0   0.0   1.0
  1.0  -1.0   0.0   0.0   0.0   1.0   0.0  -1.0   0.0
  0.0   

LoadError: KeyError: key 21 not found

In [19]:
dot(cons, )

LoadError: DimensionMismatch("one array has length 441 which does not match the length of the next one, 189.")

In [32]:
dot(transpose(n[:,:NM] .+ b[:,:NM]), cons)

LoadError: DimensionMismatch("one array has length 21 which does not match the length of the next one, 441.")

In [47]:
dot(n[:,:NM] .+ b[:,:NM], cons[19,:])

n[19,NM] + b[19,NM] + n[20,NM] + b[20,NM] + n[21,NM] + b[21,NM]

In [42]:
cons[1,:]

21-element Vector{Float64}:
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0